In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from scipy import stats
from sklearn. metrics import roc_curve, auc
import random
from matplotlib.gridspec import GridSpec
from sklearn.calibration import CalibrationDisplay
from sklearn. metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [2]:
def calculate_metrics(y_true, y_pred):
    tp = np.sum(np.logical_and(y_true == 1, y_pred == 1))
    tn = np.sum(np.logical_and(y_true == 0, y_pred == 0))
    fp = np.sum(np.logical_and(y_true == 0, y_pred == 1))
    fn = np.sum(np.logical_and(y_true == 1, y_pred == 0))
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    predictive_accuracy = (tp + tn) / (tp + tn + fp + fn)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return sensitivity, specificity, ppv, npv, predictive_accuracy, mcc

In [3]:
data = pd.read_excel("C:\\Users\\12292\\Desktop\\train size - xx (2).xlsx")

In [4]:
train_X = data.iloc[:, 1:]  # Features (excluding the first column)
train_y = data.iloc[:, 0] 

In [5]:
train_X

,BNF,PHASES,Radscore
0,1.260731,4,0.446201
1,1.385450,9,2.553235
2,1.410795,0,-1.116113
3,0.930106,0,-1.127136
4,1.066182,0,-1.669537
...,...,...,...
328,1.266495,1,-0.748227
329,1.018247,1,-1.270338
330,0.658511,5,0.754189
331,1.007611,6,-0.795665


In [6]:
train_R = data.iloc[:,3]  # Features (excluding the first column)

In [7]:
train_R

0      0.446201
1      2.553235
2     -1.116113
3     -1.127136
4     -1.669537
         ...   
328   -0.748227
329   -1.270338
330    0.754189
331   -0.795665
332    0.604769
Name: Radscore, Length: 333, dtype: float64

In [8]:
train_y 

0      1
1      1
2      0
3      0
4      0
      ..
328    0
329    0
330    1
331    0
332    1
Name: status, Length: 333, dtype: int64

In [9]:
train_P = data.iloc[:,2]  

In [10]:
data_test = pd.read_excel("C:\\Users\\12292\\Desktop\\test size - xx (2).xlsx")

In [11]:
test_y = data_test.iloc[:, 0] 

In [12]:
test_X = data_test.iloc[:, 1:4] 
test_X

,BNF,PHASES,Radscore
0,1.599177,6,-0.199217
1,1.277627,4,0.417314
2,0.932445,1,-0.781816
3,0.956438,5,-0.183729
4,1.548727,3,-0.510796
...,...,...,...
137,1.190633,4,-1.118190
138,1.035633,0,-1.140353
139,1.178697,4,1.206319
140,1.069052,0,-1.283198


In [13]:
test_X

,BNF,PHASES,Radscore
0,1.599177,6,-0.199217
1,1.277627,4,0.417314
2,0.932445,1,-0.781816
3,0.956438,5,-0.183729
4,1.548727,3,-0.510796
...,...,...,...
137,1.190633,4,-1.118190
138,1.035633,0,-1.140353
139,1.178697,4,1.206319
140,1.069052,0,-1.283198


In [14]:
test_R = data_test.iloc[:,3]  

In [15]:
test_y

0      0
1      0
2      0
3      1
4      0
      ..
137    0
138    0
139    1
140    0
141    1
Name: status, Length: 142, dtype: int64

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

rf_params = {
    'n_estimators': 210,
    'random_state':9999,
    'max_depth':3,
    'min_samples_leaf':34,
    'min_samples_split':70,
}
best_RF = RandomForestClassifier(**rf_params)

best_RF.fit(train_X, train_y)
RF_y_pred = best_RF.predict(test_X)
accuracy = accuracy_score(test_y, RF_y_pred)
print("Test Accuracy with Best Parameters:", accuracy)

Test Accuracy with Best Parameters: 0.795774647887324


In [17]:
accuracy = accuracy_score(test_y,  RF_y_pred)
RF_y_score = best_RF.predict_proba(test_X)[:, 1]
fprrf,tprrf,threshold = roc_curve(test_y,RF_y_score)
auc(fprrf,tprrf)

0.8803980099502487

In [18]:
import numpy as np  
from sklearn import metrics  
from sklearn.metrics import roc_curve, auc  
from scipy.stats import percentileofscore  

# Compute ROC curve and AUC for the original data
fprrf, tprrf, thresholds = roc_curve(test_y, RF_y_score)  
original_auc = auc(fprrf, tprrf)  
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 1000  # You can adjust this value as needed
auc_scores = []  

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):  
    # Randomly sample the test set with replacement
    bootstrap_indices = np.random.randint(0, len(test_y), size=len(test_y))  
    bootstrap_y = test_y[bootstrap_indices]  
    bootstrap_y_score = RF_y_score[bootstrap_indices]  

    # Compute AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)  
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)  
    auc_scores.append(auc_bootstrap)  

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)  
auc_scores_sorted = np.sort(auc_scores)  
n = len(auc_scores)  
lower_idx = int(np.round(n * 0.025))  # Compute the lower bound index
upper_idx = int(np.round(n * 0.975))  # Compute the upper bound index
auc_lower = auc_scores_sorted[lower_idx]  
auc_upper = auc_scores_sorted[upper_idx]  

print("Test set 95% CI for AUC: ({:.3f}, {:.3f})".format(auc_lower, auc_upper))

Original AUC: 0.880
Test set 95% CI for AUC: (0.824, 0.941)


In [19]:
from sklearn.metrics import roc_auc_score, roc_curve
RF_train_y_pred = best_RF.predict(train_X)
train_y_prob = best_RF.predict_proba(train_X)[:, 1]

train_auc = roc_auc_score(train_y, train_y_prob)
  
fprRF, tprRF, thresholds = roc_curve(train_y,train_y_prob)  
original_auc = auc(fprRF, tprRF)  
print("Original AUC: {:.3f}".format(original_auc))

Original AUC: 0.916


In [20]:
from sklearn.metrics import roc_auc_score, roc_curve
train_y_prob = best_RF.predict_proba(train_X)[:, 1]

train_auc = roc_auc_score(train_y, train_y_prob)

In [21]:
import numpy as np  
from sklearn import metrics  
from sklearn.metrics import roc_curve, auc  
from scipy.stats import percentileofscore  

# Compute original AUC
fprRF, tprRF, thresholds = roc_curve(train_y, train_y_prob)  
original_auc = auc(fprRF, tprRF)  
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 1000  # You can adjust this value as needed
auc_scores = []  

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):  
    # Randomly sample the training set with replacement
    bootstrap_indices = np.random.randint(0, len(train_y), size=len(train_y))  
    bootstrap_y = train_y[bootstrap_indices]  
    bootstrap_y_score = train_y_prob[bootstrap_indices]  

    # Compute AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)  
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)  
    auc_scores.append(auc_bootstrap)  

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)  
auc_scores_sorted = np.sort(auc_scores)  
n = len(auc_scores)  
lower_idx = int(np.round(n * 0.025))  # Compute the lower bound index
upper_idx = int(np.round(n * 0.975))  # Compute the upper bound index
auc_lower = auc_scores_sorted[lower_idx]  
auc_upper = auc_scores_sorted[upper_idx]  

print("Training set 95% CI for AUC: ({:.3f}, {:.3f})".format(auc_lower, auc_upper))

Original AUC: 0.916
Training set 95% CI for AUC: (0.884, 0.943)


In [22]:
RF_y_pred =  best_RF.predict(test_X)

In [23]:
# RF metrics
sensitivity_train, specificity_train, ppv_train, npv_train, predictive_accuracy_train, mcc_train = calculate_metrics(train_y,RF_train_y_pred)
sensitivity_test, specificity_test, ppv_test, npv_test, predictive_accuracy_test, mcc_test = calculate_metrics(test_y, RF_y_pred)

print("Train Set Metrics:")
print(f"Sensitivity: {sensitivity_train:.3f}")
print(f"Specificity: {specificity_train:.3f}")
print(f"PPV (Precision): {ppv_train:.3f}")
print(f"NPV: {npv_train:.3f}")
print(f"Predictive Accuracy: {predictive_accuracy_train:.3f}")
print(f"MCC: {mcc_train:.3f}")

print("\nTest Set Metrics:")
print(f"Sensitivity: {sensitivity_test:.3f}")
print(f"Specificity: {specificity_test:.3f}")
print(f"PPV (Precision): {ppv_test:.3f}")
print(f"NPV: {npv_test:.3f}")
print(f"Predictive Accuracy: {predictive_accuracy_test:.3f}")
print(f"MCC: {mcc_test:.3f}")

Train Set Metrics:
Sensitivity: 0.823
Specificity: 0.844
PPV (Precision): 0.807
NPV: 0.858
Predictive Accuracy: 0.835
MCC: 0.666

Test Set Metrics:
Sensitivity: 0.746
Specificity: 0.840
PPV (Precision): 0.806
NPV: 0.787
Predictive Accuracy: 0.796
MCC: 0.590


In [24]:
#Brier is at the bottom

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
best_model_XGB = xgb.XGBClassifier(n_estimators=254, learning_rate=0.0395, max_depth=3, min_child_weight=7,colsample_bytree=0.4, 
                                   subsample=0.4,random_state=527)


best_model_XGB.fit(train_X, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0395, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              n_estimators=254, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=527, ...)

In [26]:
XGB_train_y_pred = best_model_XGB.predict(train_X)
XGBboost_train_y_score = best_model_XGB.predict_proba(train_X)[:, 1]
fprXGB,tprXGB,threshold = roc_curve(train_y,XGBboost_train_y_score)
auc(fprXGB,tprXGB)

0.9128081340062908

In [27]:
XGB_y_pred = best_model_XGB.predict(test_X)
XGBboost_y_score = best_model_XGB.predict_proba(test_X)[:, 1]
fprxgb,tprxgb,threshold = roc_curve(test_y,XGBboost_y_score)
auc(fprxgb,tprxgb)

0.8877611940298507

In [28]:
from sklearn.metrics import roc_auc_score, roc_curve
train_y_prob =best_model_XGB.predict_proba(train_X)[:, 1]

train_auc = roc_auc_score(train_y, train_y_prob)

In [29]:
# XGB metrics
sensitivity_train, specificity_train, ppv_train, npv_train, predictive_accuracy_train, mcc_train = calculate_metrics(train_y,XGB_train_y_pred)
sensitivity_test, specificity_test, ppv_test, npv_test, predictive_accuracy_test, mcc_test = calculate_metrics(test_y,XGB_y_pred)

print("Train Set Metrics:")
print(f"Sensitivity: {sensitivity_train:.4f}")
print(f"Specificity: {specificity_train:.4f}")
print(f"PPV (Precision): {ppv_train:.4f}")
print(f"NPV: {npv_train:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_train:.4f}")
print(f"MCC: {mcc_train:.4f}")

print("\nTest Set Metrics:")
print(f"Sensitivity: {sensitivity_test:.4f}")
print(f"Specificity: {specificity_test:.4f}")
print(f"PPV (Precision): {ppv_test:.3f}")
print(f"NPV: {npv_test:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_test:.4f}")
print(f"MCC: {mcc_test:.4f}")

Train Set Metrics:
Sensitivity: 0.8367
Specificity: 0.8441
PPV (Precision): 0.8092
NPV: 0.8674
Predictive Accuracy: 0.8408
MCC: 0.6787

Test Set Metrics:
Sensitivity: 0.7612
Specificity: 0.8667
PPV (Precision): 0.836
NPV: 0.8025
Predictive Accuracy: 0.8169
MCC: 0.6332


In [35]:
import numpy as np  
from sklearn import metrics  
from sklearn.metrics import roc_curve, auc  
from scipy.stats import percentileofscore  

# Compute ROC curve and AUC for the original data
fprxgb, tprxgb, thresholds = roc_curve(test_y, XGBboost_y_score)  
original_auc = auc(fprxgb, tprxgb)  
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 1000  # You can adjust this value as needed
auc_scores = []  

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):  
    # Randomly sample the test set with replacement
    bootstrap_indices = np.random.randint(0, len(test_y), size=len(test_y))  
    bootstrap_y = test_y[bootstrap_indices]  
    bootstrap_y_score = XGBboost_y_score[bootstrap_indices]  

    # Compute AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)  
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)  
    auc_scores.append(auc_bootstrap)  

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)  
auc_scores_sorted = np.sort(auc_scores)  
n = len(auc_scores)  
lower_idx = int(np.round(n * 0.025))  # Compute the lower bound index
upper_idx = int(np.round(n * 0.975))  # Compute the upper bound index
auc_lower = auc_scores_sorted[lower_idx]  
auc_upper = auc_scores_sorted[upper_idx]  

print("Test set 95% CI for AUC: ({:.3f}, {:.3f})".format(auc_lower, auc_upper))

Original AUC: 0.888
Test set 95% CI for AUC: (0.831, 0.946)


In [33]:
import numpy as np  
from sklearn import metrics  
from sklearn.metrics import roc_curve, auc  
from scipy.stats import percentileofscore  

# Compute original AUC
fprXGB, tprXGB, thresholds = roc_curve(train_y, XGBboost_train_y_score)  
original_auc = auc(fprXGB, tprXGB)  
print("Original AUC: {:.3f}".format(original_auc))

# Bootstrapping parameters
n_bootstraps = 1000  # You can adjust this value as needed
auc_scores = []  

# Bootstrapping process
np.random.seed(321)  # Set random seed for reproducibility
for _ in range(n_bootstraps):  
    # Randomly sample the training set with replacement
    bootstrap_indices = np.random.randint(0, len(train_y), size=len(train_y))  
    bootstrap_y = train_y[bootstrap_indices]  
    bootstrap_y_score = XGBboost_train_y_score[bootstrap_indices]  

    # Compute AUC for each bootstrap sample
    fpr_bootstrap, tpr_bootstrap, _ = roc_curve(bootstrap_y, bootstrap_y_score)  
    auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)  
    auc_scores.append(auc_bootstrap)  

# Calculate the 95% confidence interval
auc_scores = np.array(auc_scores)  
auc_scores_sorted = np.sort(auc_scores)  
n = len(auc_scores)  
lower_idx = int(np.round(n * 0.025))  # Compute the lower bound index
upper_idx = int(np.round(n * 0.975))  # Compute the upper bound index
auc_lower = auc_scores_sorted[lower_idx]  
auc_upper = auc_scores_sorted[upper_idx]  

print("Training set 95% CI for AUC: ({:.3f}, {:.3f})".format(auc_lower, auc_upper))

Original AUC: 0.913
Training set 95% CI for AUC: (0.881, 0.941)


In [36]:
from sklearn.ensemble import GradientBoostingClassifier

In [37]:
    best_GB = GradientBoostingClassifier(learning_rate=0.005,  
                                           n_estimators=278,  
                                           min_samples_split=38,  
                                           min_samples_leaf=27,  
                                           max_depth=3,  
                                           max_features=2,  
                                           subsample=0.7,
                                           random_state=102030)  
      

    best_GB.fit(train_X, train_y)  

GradientBoostingClassifier(learning_rate=0.005, max_features=2,
                           min_samples_leaf=27, min_samples_split=38,
                           n_estimators=278, random_state=102030,
                           subsample=0.7)

In [38]:
test_gb_pred_proba = best_GB.predict_proba(test_X)[:, 1]
test_auc = roc_auc_score(test_y, test_gb_pred_proba)
GBboost_y_pred=best_GB.predict(test_X)
print("testAUC:", test_auc)
fprgb,tprgb,threshold = roc_curve(test_y,test_gb_pred_proba)
auc(fprgb,tprgb)

testAUC: 0.8906467661691542


0.8906467661691542

In [39]:
fprgb,tprgb,threshold = roc_curve(test_y,test_gb_pred_proba)
auc(fprgb,tprgb)

0.8906467661691542

In [43]:
import numpy as np
from sklearn.metrics import roc_auc_score

# Set random seed
np.random.seed(321)

# Set the number of bootstrap resampling iterations
n_bootstraps = 1000

# Store the AUC values calculated for each bootstrap sample
auc_scores = []

# Perform bootstrap resampling
for _ in range(n_bootstraps):
    # Draw samples with replacement from the test data
    indices = np.random.choice(len(test_y), len(test_y), replace=True)
    bootstrap_test_pred_proba = test_gb_pred_proba[indices]
    bootstrap_test_y = test_y[indices]
    
    # Calculate AUC and store it
    bootstrap_auc = roc_auc_score(bootstrap_test_y, bootstrap_test_pred_proba)
    auc_scores.append(bootstrap_auc)

# Calculate the lower and upper bounds of the confidence interval
lower_bound = np.percentile(auc_scores, 2.5)
upper_bound = np.percentile(auc_scores, 97.5)

print("95% confidence interval: [{:.4f}, {:.4f}]".format(lower_bound, upper_bound))

95% confidence interval: [0.8360, 0.9487]


In [44]:
GBboost_train_y_pred=best_GB.predict(train_X)
train_GB_pred_proba = best_GB.predict_proba(train_X)[:, 1]
train_auc = roc_auc_score(train_y, train_GB_pred_proba)
print("Train AUC:", train_auc )

Train AUC: 0.9233230926779313


In [45]:
fprGB,tprGB,threshold = roc_curve(train_y,train_GB_pred_proba)
auc(fprGB,tprGB)

0.9233230926779313

In [46]:
import numpy as np
from sklearn.metrics import roc_auc_score

# Set random seed
np.random.seed(321)

# Set the number of bootstrap resampling iterations
n_bootstraps = 1000

# Store the AUC values calculated for each bootstrap sample
auc_scores = []

# Perform bootstrap resampling
for _ in range(n_bootstraps):
    # Draw samples with replacement from the training data
    indices = np.random.choice(len(train_y), len(train_y), replace=True)
    bootstrap_train_pred_proba = train_GB_pred_proba[indices]
    bootstrap_train_y = train_y[indices]
    
    # Calculate AUC and store it
    bootstrap_auc = roc_auc_score(bootstrap_train_y, bootstrap_train_pred_proba)
    auc_scores.append(bootstrap_auc)

# Calculate the lower and upper bounds of the confidence interval
lower_bound = np.percentile(auc_scores, 2.5)
upper_bound = np.percentile(auc_scores, 97.5)

print("95% confidence interval: [{:.4f}, {:.4f}]".format(lower_bound, upper_bound))


95% confidence interval: [0.8951, 0.9482]


In [48]:
# GBM metrics
sensitivity_train, specificity_train, ppv_train, npv_train, predictive_accuracy_train, mcc_train = calculate_metrics(train_y,GBboost_train_y_pred)
sensitivity_test, specificity_test, ppv_test, npv_test, predictive_accuracy_test, mcc_test = calculate_metrics(test_y,GBboost_y_pred)

print("Train Set Metrics:")
print(f"Sensitivity: {sensitivity_train:.4f}")
print(f"Specificity: {specificity_train:.4f}")
print(f"PPV (Precision): {ppv_train:.4f}")
print(f"NPV: {npv_train:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_train:.4f}")
print(f"MCC: {mcc_train:.4f}")

print("\nTest Set Metrics:")
print(f"Sensitivity: {sensitivity_test:.4f}")
print(f"Specificity: {specificity_test:.4f}")
print(f"PPV (Precision): {ppv_test:.3f}")
print(f"NPV: {npv_test:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_test:.4f}")
print(f"MCC: {mcc_test:.4f}")

Train Set Metrics:
Sensitivity: 0.8367
Specificity: 0.8495
PPV (Precision): 0.8146
NPV: 0.8681
Predictive Accuracy: 0.8438
MCC: 0.6844

Test Set Metrics:
Sensitivity: 0.7463
Specificity: 0.8533
PPV (Precision): 0.820
NPV: 0.7901
Predictive Accuracy: 0.8028
MCC: 0.6047


In [49]:
best_GB_y_score =best_GB.predict_proba(test_X)[:, 1]

In [50]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV


best_params_LGBM = {
    'lambda_l1': 0.7,
    'lambda_l2': 1e-05,
    'learning_rate': 0.005,
    'n_estimators': 600,
    'max_depth': 4,
    'num_leaves': 12,
    'max_bin': 14,
    'min_data_in_leaf': 32,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.6,
    'bagging_freq': 10,
    'random_state': 123,
    'verbose': -1
}


best_LGBM = lgb.LGBMClassifier(**best_params_LGBM)
best_LGBM.fit(train_X, train_y)

LGBMClassifier(bagging_fraction=0.6, bagging_freq=10, feature_fraction=0.9,
               lambda_l1=0.7, lambda_l2=1e-05, learning_rate=0.005, max_bin=14,
               max_depth=4, min_data_in_leaf=32, n_estimators=600,
               num_leaves=12, random_state=123, verbose=-1)

In [51]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
train_y_pred = best_LGBM.predict(train_X)
train_accuracy = accuracy_score(train_y, train_y_pred)
print("Training Accuracy:", train_accuracy)
train_y_prob = best_LGBM.predict_proba(train_X)[:, 1]
train_auc = roc_auc_score(train_y, train_y_prob)
print("Training AUC：", train_auc)

Training Accuracy: 0.8378378378378378
Training AUC： 0.9172701338599956


In [52]:
y_score_LGBM = best_LGBM.predict_proba(test_X)[:, 1]
LGBM_y_pred=best_LGBM.predict(test_X)
test_accuracy = accuracy_score(test_y, LGBM_y_pred)
print("Test_accuracy:", test_accuracy)
fprlgbm, tprlgbm, threshold = roc_curve(test_y, y_score_LGBM)
test_auc = roc_auc_score(test_y, y_score_LGBM )

print("Test AUC:", test_auc)

Test_accuracy: 0.823943661971831
Test AUC: 0.8920398009950249


In [53]:
fprLGBM, tprLGBM, thresholds= roc_curve(train_y, train_y_prob)
fprlgbm, tprlgbm, thresholds = roc_curve(test_y, y_score_LGBM)

In [54]:
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score

np.random.seed(321)

n_bootstraps = 1000
auc_scores_train = []
auc_scores_test = []

for i in range(n_bootstraps):
    # Perform bootstrap resampling on the training set
    train_y_sampled, train_y_prob_sampled = resample(train_y, train_y_prob)
    
    # Calculate AUC for the resampled training set
    auc_scores_train.append(roc_auc_score(train_y_sampled, train_y_prob_sampled))
    
    # Perform bootstrap resampling on the validation set
    test_y_sampled, y_score_LGBM_sampled = resample(test_y, y_score_LGBM)
    
    # Calculate AUC for the resampled validation set
    auc_scores_test.append(roc_auc_score(test_y_sampled, y_score_LGBM_sampled))

# Calculate the lower and upper bounds of the confidence interval for the training set
lower_bound_train = np.percentile(auc_scores_train, 2.5)
upper_bound_train = np.percentile(auc_scores_train, 97.5)

# Calculate the lower and upper bounds of the confidence interval for the validation set
lower_bound_test = np.percentile(auc_scores_test, 2.5)
upper_bound_test = np.percentile(auc_scores_test, 97.5)

print("95% confidence interval for the training set: [{:.4f}, {:.4f}]".format(lower_bound_train, upper_bound_train))
print("95% confidence interval for the validation set: [{:.4f}, {:.4f}]".format(lower_bound_test, upper_bound_test))


95% confidence interval for the training set: [0.8873, 0.9452]
95% confidence interval for the validation set: [0.8250, 0.9429]


In [57]:
# LGBM metrics
sensitivity_train, specificity_train, ppv_train, npv_train, predictive_accuracy_train, mcc_train = calculate_metrics(train_y, train_y_pred)
sensitivity_test, specificity_test, ppv_test, npv_test, predictive_accuracy_test, mcc_test = calculate_metrics(test_y, LGBM_y_pred)

print("Train Set Metrics:")
print(f"Sensitivity: {sensitivity_train:.4f}")
print(f"Specificity: {specificity_train:.4f}")
print(f"PPV (Precision): {ppv_train:.4f}")
print(f"NPV: {npv_train:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_train:.4f}")
print(f"MCC: {mcc_train:.4f}")

print("\nTest Set Metrics:")
print(f"Sensitivity: {sensitivity_test:.4f}")
print(f"Specificity: {specificity_test:.4f}")
print(f"PPV (Precision): {ppv_test:.3f}")
print(f"NPV: {npv_test:.4f}")
print(f"Predictive Accuracy: {predictive_accuracy_test:.4f}")
print(f"MCC: {mcc_test:.4f}")

Train Set Metrics:
Sensitivity: 0.8231
Specificity: 0.8495
PPV (Precision): 0.8121
NPV: 0.8587
Predictive Accuracy: 0.8378
MCC: 0.6717

Test Set Metrics:
Sensitivity: 0.7761
Specificity: 0.8667
PPV (Precision): 0.839
NPV: 0.8125
Predictive Accuracy: 0.8239
MCC: 0.6470


In [56]:
from sklearn.metrics import brier_score_loss

# Brier 
LGBM_brier_score = brier_score_loss(test_y, y_score_LGBM)
GB_brier_score = brier_score_loss(test_y, test_gb_pred_proba)
XGB_brier_score = brier_score_loss(test_y, XGBboost_y_score)
RF_brier_score = brier_score_loss(test_y, RF_y_score)

print("LGBM Brier score:", LGBM_brier_score)
print("GB Brier score:", GB_brier_score)
print("XGB Brier score:", XGB_brier_score)
print("RF Brier score:", RF_brier_score)

LGBM Brier score: 0.1320138283556927
GB Brier score: 0.1427594910326072
XGB Brier score: 0.13496849132066688
RF Brier score: 0.14742283265438752
